In [1]:
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split 

import warnings
warnings.filterwarnings("ignore")

# PUF

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue May 21 08:57:08 2019

@author: stjepan
"""

n_stages = 16
n_crp = 1000

stage_delays = np.random.normal(size=n_stages+1)
# print(stage_delays)

x = []
y = []

for _ in range(n_crp):
    challenge_vector = np.random.randint(2, size=n_stages)
    # print(challenge_vector)
    feature_vector = []
    for i in range(n_stages):
        feature = 1
        for j in range(i, n_stages):
            feature = feature * pow(-1, challenge_vector[j])
        feature_vector.append(feature)
    feature_vector.append(1)
    # print(feature_vector)
    puf_result = np.dot(stage_delays, feature_vector) > 0
    x.append(feature_vector)
    y.append(puf_result)

x_array = np.array(x)
y_array = np.array(y)
# print(x_array.shape)
# print(y_array.shape)

X_train, X_test, Y_train, Y_test = train_test_split(
    x_array, y_array, test_size=0.2, random_state=0)
# print(X_train.shape)

print("- Logistic Classifier:")
clf_lr = LogisticRegression(solver="lbfgs").fit(X_train, Y_train)
print("\t traning accuracy:", clf_lr.score(X_train, Y_train))
Y_pred = clf_lr.predict(X_test)
print("\t testing accuracy:", clf_lr.score(X_test, Y_test))
print()

print("- Multi-layer Perceptron:")
clf_mlp = MLPClassifier(hidden_layer_sizes=(n_stages,)).fit(X_train, Y_train)
print("\t traning accuracy:", clf_mlp.score(X_train, Y_train))
Y_pred = clf_mlp.predict(X_test)
print("\t testing accuracy:", clf_mlp.score(X_test, Y_test))
print()

- Logistic Classifier:
	 traning accuracy: 0.99125
	 testing accuracy: 0.995

- Multi-layer Perceptron:
	 traning accuracy: 0.9925
	 testing accuracy: 0.99



# XOR PUF

In [3]:
n_stages = 8
n_puf = 3
n_crp = 10000

stage_delays = np.random.normal(size=(n_puf, n_stages+1))

x = []
y = []

for _ in range(n_crp):

    challenge_vector = np.random.randint(2, size=(n_puf, n_stages))
    feature_vector_xor = []
    puf_result = 0

    for k in range(n_puf):
        '''
        for each puf, 
        calculate its result and xor with the previous one
        '''
        feature_vector = []
        for i in range(n_stages):
            feature = 1
            for j in range(i, n_stages):
                feature = feature * pow(-1, challenge_vector[k, j])
            feature_vector.append(feature)
        feature_vector.append(1)
        puf_result ^= np.dot(stage_delays[k], feature_vector) > 0  # XOR Arbiter
#         print(puf_result, stage_delays[k], feature_vector)
        feature_vector_xor.append(feature_vector)

    x.append(feature_vector_xor)
    y.append(puf_result)

x_array = np.array(x)
y_array = np.array(y)
# print(x_array.shape)
# print(y_array.shape)

X_train, X_test, Y_train, Y_test = train_test_split(
    x_array.reshape(x_array.shape[0], x_array.shape[1]*x_array.shape[2]), y_array, test_size=0.2, random_state=0)
# print(X_train.shape)

print("- Logistic Classifier:")
clf_lr = LogisticRegression(solver="lbfgs").fit(X_train, Y_train)
print("\t traning accuracy:", clf_lr.score(X_train, Y_train))
Y_pred = clf_lr.predict(X_test)
print("\t testing accuracy:", clf_lr.score(X_test, Y_test))
print()

print("- Multi-layer Perceptron:")
clf_mlp = MLPClassifier(hidden_layer_sizes=(
    n_stages, n_puf, 2), activation='relu').fit(X_train, Y_train)
print("\t traning accuracy:", clf_mlp.score(X_train, Y_train))
Y_pred = clf_mlp.predict(X_test)
print("\t testing accuracy:", clf_mlp.score(X_test, Y_test))
print()

- Logistic Classifier:
	 traning accuracy: 0.567625
	 testing accuracy: 0.5565

- Multi-layer Perceptron:
	 traning accuracy: 0.979875
	 testing accuracy: 0.9705



# Interpose PUF